#### Operating Instructions: ####
## IT IS IMPORTANT TO FOLLOW THE ORDER OF EXECUTION INDICATED, OTHERWISE THE MODEL WILL NOT WORK.##
**First Step**:  Load required libraries;
**Second Step**:  Execute the set of "Secondary Functions" cells of interest;
**Third Step**:  Execute the Main script cell of interest;
**Final Step**: Execute the "Downloading vtk files generated" cell

# **Case Study V** - Multicellular systems, the role of elastic mismatch and cell motility (pages 81-97)

In [ ]:
# Importing and loading libraries used:
from scipy import *
from scipy import sparse
from scipy.linalg import toeplitz
from math import *
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import scipy.sparse as sparse
from datetime import date
from datetime import datetime
import time
import random

## Secondary functions:

In [ ]:
# Defining functions required:
## FUNCTION 1 --> micro_poly_cell

## This function introduces randomly distributed and slightly overlapping cells 
## having radius of R into the simulation cell

def micro_poly_cell(Nx,Ny,ncell,R):
  #global xc
  #global yc
  # Initialize the order parameter of the cells:
  ## Longhand initialization
  #for icell in range (1,ncell):
    #for i in range (1,Nx):
      #for j in range (1,Ny):
         #phis(i,j,icell) == 0.0
  phis = np.zeros(shape=(Nx,Ny,ncell)) # Initialization for optimization, 3-D ARRAY
  #phis = zeros(Nx,Ny,ncell)
  # end of initialize?

  ###################### Generating slightly overlapping cell microstructure, if ncell > 2 (START) ######################
  if(ncell == 30):
    R2 = 2.0*R
    Rsq = R*R
    ncell = 0
    xmin = 0.0
    ymin = 0.0
    xmax = Nx
    ymax = Ny
    xc = np.zeros((64,64))
    yc = np.zeros((64,64))

    for iter in range(1,500001):
      xnc = Nx*np.random.random_sample() # Coordinates of cell centers (assigned randomly)
      ync = Ny*np.random.random_sample() # Coordinates of cell centers (assigned randomly)

      iflag = 1

      if(((xnc-R) < xmin) or ((xnc+R) > xmax)):
        iflag = 0
      
      if(((ync-R) < ymin) or ((ync+R) > ymax)):
        iflag = 0
      
      if(iflag == 1):
        for i in range(1,ncell+1):
          inds = i-1
          xdist = sqrt((xc[np.unravel_index(inds, xc.shape, 'F')]-xnc)**2 +
                       (yc[np.unravel_index(inds, yc.shape, 'F')]-ync)**2)
          #xdist = sqrt((xc[inds]-xnc)**2 + (yc[inds]-ync)**2)
          #xdist = sqrt((xc(i)-xnc)^2 +(yc(i)-ync)^2) # Linear Indexing (in matrixes): https://es.mathworks.com/company/newsletters/articles/matrix-indexing-in-matlab.html

          if(xdist <= R*1.6):
            iflag = 0

      if(iflag == 1):
        ncell = ncell + 1

        inds_1 = ncell - 1
        xc[np.unravel_index(inds_1, xc.shape, 'F')] = xnc
        yc[np.unravel_index(inds_1, yc.shape, 'F')] = ync
        #xc[(64-ncell), 1] = xnc
        #yc[(64-ncell), 1] = ync
        #xc(ncell) = xnc
        #yc(ncell) = ync

        for i in range(1,Nx+1):
          for j in range(1,Ny+1):

            if((i-xnc)**2 + (j-ync)**2 < Rsq):
              #phis(i,j,ncell) == 0.999 # ES CORRECTO?? PARECE QUE NO!
              phis[i-1,j-1,ncell-1] = 0.999 # MI CORRECCION
      
      if(ncell == 30):
        break

    ## Randomize cell self propulsion velocities: CAMBIAR ESTO, AQUI HAY QUE METER QUE CREZCAN EN RADIO
    velc = 0

    #for i in range(1,ncell+1):
      #ix = np.random.random_sample()
      #vac = np.zeros(ncell)
      #if(ix <= 0.5):
        #vac[i-1] = -velc
        #vac[np.unravel_index(inds, xc.shape, 'F')] == -velc
      #else:
        #vac[i-1] = velc
        #vac[np.unravel_index(inds, xc.shape, 'F')] == velc
    
    
    print("iteration done:%5d\n" % (iter))
    print("number of cell created:%5d\n" % (ncell))
    #print("Iteration done: \n" + str(iter))
    #print("Number of cells created: \n" + str(ncell))

    ## Soft cells: DEJAR ESTO?? MODIFICARLO??
    nccel = int(ncell/2) # NUMBER OF SOFT CELLS
    ccell = np.zeros(nccel)
    # Creating a list of random numbers without duplicates: https://stackoverflow.com/questions/9755538/how-do-i-create-a-list-of-random-numbers-without-duplicates
    ccell = random.sample(range(1,ncell+1),nccel)
    ccell = np.array(ccell)

    # end if ncell = 30

  ###################### Generating slightly overlapping cell microstructure, if ncell > 2 (END) ######################


  ###################### 2-cell simulation ###################### 

  if(ncell == 2):
    nccel = 0
    ccell=np.zeros(1)
    ccell[0] = 10000

    R2 = R*R

    xc = np.zeros((64,64))
    yc = np.zeros((64,64))
    xc[0,0] = Nx/2-1.25*R
    yc[0,0] = Ny/2

    xc[0,1] = Nx/2+1.25*R
    yc[0,1] = Ny/2

    ncol = 1
    nrow = 2
    
    icell = 0
    
    for icol in range(1,ncol+1):

      for irow in range(1,nrow+1):
        
        icell = icell + 1
        
        dx = xc[icol-1,irow-1]
        dy = yc[icol-1,irow-1]

        for i in range(1,Nx+1):
          for j in range(1,Ny+1):

            if((i- dx)*(i-dx) + (j-dy)*(j-dy) < R2):
              phis[i-1,j-1,icell-1] = 0.999

    ## Cell self propulsion velocity:

    vac = np.zeros(2)
    vac[0] = 0.5
    vac[1] = -0.5
    
    # end if ncell = 2

  return ncell,phis,nccel,ccell

if __name__ == '__main__':
    pass
    
    ###################### ending function ######################


In [ ]:
## laplacian:

# This function generates the Laplace operator by using five-point stencil with periodic boundaries.

import scipy as sp
from scipy import linalg
from scipy import sparse
import numpy as np
import math as m
    
def laplacian(Nx,Ny,dx,dy):

    format('long')
    nxny=Nx*Ny # Total number of grid points in the simulation cell

    ## Generate Laplace operator for five-point stencil:
    r=np.zeros(shape=(1,Nx))
    r[:,:2] = 2,-1
    T=sp.linalg.toeplitz(r) # symmetric Toeplitz matrix where: If r is a real vector, then r defines the first row of the matrix.
    E=sp.sparse.eye(Nx) # Sparse matrix (nx X nx) with ones on diagonal
    grad= -(sparse.kron(T,E) + sparse.kron(E,T)) # Computes the Kronecker product, a composite array made of blocks of the second array scaled by the first (POR AQUI ANDA EL FALLO)
    
    ## Modify the stencil for periodic boundaries:  
    for i in range(1,Nx+1):
        ii = (i-1)*Nx + 1
        jj = ii + Nx - 1
        grad[ii-1,jj-1]=1.0
        grad[jj-1,ii-1]=1.0
        
        kk = nxny - Nx + i
        grad[i-1,kk-1]=1.0
        grad[kk-1,i-1]=1.0
    
    ## Introduce dimensionality to the stencil:
    grad = grad/(dx*dy)
    
    return grad
    
if __name__ == '__main__':
    pass
    

In [ ]:
## vec2matx

# This function turns one-dimensional array to a square [N,N] matrix. 
# It is based on the built-in function vect2mat in Matlab/Octave and also utilizes two other built-in functions ceil and reshape.

import numpy as np
import math as m
    
def vec2matx(V=None,N=None,*args,**kwargs):

  format('long')
  V = np.transpose(V)
  V = np.ravel(V)
  R = m.ceil(len(V)/N)
  #M = np.reshape(np.transpose(V, (N,R), order='F'))
  M = np.reshape(V, (N,R), order='F').transpose() # FUNCIONA EN TEORIA!!??
  return M
    
if __name__ == '__main__':
    pass
    

In [ ]:
## free_energy_v2:

# This function evaluates the functional derivative of free energy function (Eq. 4.59)
# at all grid points for the current cell under consideration in the main program

import numpy as np

def free_energy_v2(Nx,Ny,icell,ncell,gamma_cell,kappa,phi,phis):

  format('long')
  sum_phi = np.zeros(shape=(Nx,Ny)) # Initialize sum_phi for all grid points
  
  # Go over total number of cells in the simulation if jcell counter is not equal to icell add their value to sum_phi for all grid points
  for jcell in range(1,ncell+1):
    if (icell != jcell):
      sum_phi = sum_phi + (phis[:,:,jcell-1])**2 # MI VERSION
      #sum_phi = sum_phi + phis(np.arange(),np.arange(),jcell)**2 # RESULTADO DEL SMOP
      #sum_phi = sum_phi + phis(np.power( range(), range(),jcell,2)) # REVISAR ESTA LINEA
  
  # Calculate the functional derivative of the free energy for the current cell, the terms in the first bracket, Eq. 5.4.7, at all grid points
  dfdphi = gamma_cell*phi*(1.0-phi)*(1.0-2.0*phi) + 2.0*kappa*phi*sum_phi # EN PRINCIPIO ESTA BIEN
  
  return dfdphi
    
if __name__ == '__main__':
    pass

In [ ]:
## gradient_mat:

## This function calculates the one-dimensional gradients for x and y directions of a given matrix.
## Function utilizes the built-in function gradient in Matlab/Octave, returns the values that are adjusted for grid spacing and the periodic boundaries
    
def gradient_mat(matx,Nx,Ny,dx,dy):

    format('long')
    
    matdx,matdy= np.gradient(matx)

    # for periodic boundaries:

    matdx[:Nx,0] = 0.5*(matx[:Nx,1] - matx[:Nx,Nx-1])
    matdx[:Nx,Nx-1] = 0.5*(matx[:Nx,0] - matx[:Nx,Nx-2])

    matdy[0,:Ny] = 0.5*(matx[1,:Ny] - matx[Ny-1,:Ny])
    matdy[Ny-1,:Ny] = 0.5*(matx[0,:Ny] - matx[Ny-2,:Ny])

    matdx = 2.0*matdx/dx
    matdy = 2.0*matdy/dy
    
    return matdx,matdy
    
if __name__ == '__main__':
    pass
    

In [ ]:
## write_vtk_grid_values

# This function writes the grid point values to a vtk file to generate the contour plots to be viewed by using Paraview.
# This function and its slightly modified versions are used throughout the book for visualization and animation of the results.
# Although, Matlab and Octave have impressive graphic routines; however, Paraview has more powerful features for visualization and spatial and temporal analysis of the data.

def write_vtk_grid_values(Nx=None,Ny=None,dx=None,dy=None,istep=None,data1=None,*args,**kwargs):

  format('long')
  #-- open output file
  
  fname = open("time_%d.vtk" % (istep), "w+") # Creating the file
  out = open("time_%d.vtk" % (istep), "a") # Appending text in the file

  nz = 1
  
  npoin = Nx*Ny*nz

  # start writing ASCII VTK file:
  
  # header of VTK file
  
  out.write("# vtk DataFile Version 2.0\n")
  out.write("time_10.vtk\n")
  out.write("ASCII\n")
  out.write("DATASET STRUCTURED_GRID\n")

  #--- coords of grid points:

  out.write("DIMENSIONS %5d %5d %5d\n" % (Nx,Ny,nz))

  out.write("POINTS %7d float\n" % (npoin))
  for i in range(1,Nx+1):
    for j in range(1,Ny+1):
      x =(i-1)*dx
      y =(j-1)*dy
      z = 0.0
      
      out.write("%14.6e %14.6e %14.6e\n" % (x,y,z))

  #--- write grid point values:

  out.write("POINT_DATA %5d\n" % (npoin))
  out.write("SCALARS CON float 1\n")
  out.write("LOOKUP_TABLE default\n")

  for i in range(1,Nx+1):
    for j in range(1,Ny+1):
      ii=(i-1)*Nx+j
      
      out.write("%14.6e\n" % (data1[i-1,j-1]))

  out.close()
  return
    
if __name__ == '__main__':
  pass
    

## Secondary functions (Changing kappa):

In [ ]:
# Defining functions required:
## FUNCTION 1 --> micro_poly_cell

## This function introduces randomly distributed and slightly overlapping cells 
## having radius of R into the simulation cell

def micro_poly_cell(Nx,Ny,ncell,R):
  #global xc
  #global yc
  # Initialize the order parameter of the cells:
  ## Longhand initialization
  #for icell in range (1,ncell):
    #for i in range (1,Nx):
      #for j in range (1,Ny):
         #phis(i,j,icell) == 0.0
  phis = np.zeros(shape=(Nx,Ny,ncell)) # Initialization for optimization, 3-D ARRAY
  #phis = zeros(Nx,Ny,ncell)
  # end of initialize?

  ###################### Generating slightly overlapping cell microstructure, if ncell > 2 (START) ######################
  if(ncell == 30):
    R2 = 2.0*R
    Rsq = R*R
    ncell = 0
    xmin = 0.0
    ymin = 0.0
    xmax = Nx
    ymax = Ny
    xc = np.zeros((64,64))
    yc = np.zeros((64,64))

    for iter in range(1,500001):
      xnc = Nx*np.random.random_sample() # Coordinates of cell centers (assigned randomly)
      ync = Ny*np.random.random_sample() # Coordinates of cell centers (assigned randomly)

      iflag = 1

      if(((xnc-R) < xmin) or ((xnc+R) > xmax)):
        iflag = 0
      
      if(((ync-R) < ymin) or ((ync+R) > ymax)):
        iflag = 0
      
      if(iflag == 1):
        for i in range(1,ncell+1):
          inds = i-1
          xdist = sqrt((xc[np.unravel_index(inds, xc.shape, 'F')]-xnc)**2 +
                       (yc[np.unravel_index(inds, yc.shape, 'F')]-ync)**2)
          #xdist = sqrt((xc[inds]-xnc)**2 + (yc[inds]-ync)**2)
          #xdist = sqrt((xc(i)-xnc)^2 +(yc(i)-ync)^2) # Linear Indexing (in matrixes): https://es.mathworks.com/company/newsletters/articles/matrix-indexing-in-matlab.html

          if(xdist <= R*1.6):
            iflag = 0

      if(iflag == 1):
        ncell = ncell + 1

        inds_1 = ncell - 1
        xc[np.unravel_index(inds_1, xc.shape, 'F')] = xnc
        yc[np.unravel_index(inds_1, yc.shape, 'F')] = ync
        #xc[(64-ncell), 1] = xnc
        #yc[(64-ncell), 1] = ync
        #xc(ncell) = xnc
        #yc(ncell) = ync

        for i in range(1,Nx+1):
          for j in range(1,Ny+1):

            if((i-xnc)**2 + (j-ync)**2 < Rsq):
              #phis(i,j,ncell) == 0.999 # ES CORRECTO?? PARECE QUE NO!
              phis[i-1,j-1,ncell-1] = 0.999 # MI CORRECCION
      
      if(ncell == 30):
        break

    ## Randomize cell self propulsion velocities: CAMBIAR ESTO, AQUI HAY QUE METER QUE CREZCAN EN RADIO
    velc = 0

    #for i in range(1,ncell+1):
      #ix = np.random.random_sample()
      #vac = np.zeros(ncell)
      #if(ix <= 0.5):
        #vac[i-1] = -velc
        #vac[np.unravel_index(inds, xc.shape, 'F')] == -velc
      #else:
        #vac[i-1] = velc
        #vac[np.unravel_index(inds, xc.shape, 'F')] == velc
    
    
    print("iteration done:%5d\n" % (iter))
    print("number of cell created:%5d\n" % (ncell))
    #print("Iteration done: \n" + str(iter))
    #print("Number of cells created: \n" + str(ncell))

    ## Soft cells: DEJAR ESTO?? MODIFICARLO??
    nccel = int(ncell/2) # NUMBER OF SOFT CELLS
    ccell = np.zeros(nccel)
    # Creating a list of random numbers without duplicates: https://stackoverflow.com/questions/9755538/how-do-i-create-a-list-of-random-numbers-without-duplicates
    ccell = random.sample(range(1,ncell+1),nccel)
    ccell = np.array(ccell)

    # end if ncell = 30

  ###################### Generating slightly overlapping cell microstructure, if ncell > 2 (END) ######################


  ###################### 2-cell simulation ###################### 

  if(ncell == 2):
    nccel = 0
    ccell=np.zeros(1)
    ccell[0] = 10000

    R2 = R*R

    xc = np.zeros((64,64))
    yc = np.zeros((64,64))
    xc[0,0] = Nx/2-1.25*R
    yc[0,0] = Ny/2

    xc[0,1] = Nx/2+1.25*R
    yc[0,1] = Ny/2

    ncol = 1
    nrow = 2
    
    icell = 0
    
    for icol in range(1,ncol+1):

      for irow in range(1,nrow+1):
        
        icell = icell + 1
        
        dx = xc[icol-1,irow-1]
        dy = yc[icol-1,irow-1]

        for i in range(1,Nx+1):
          for j in range(1,Ny+1):

            if((i- dx)*(i-dx) + (j-dy)*(j-dy) < R2):
              phis[i-1,j-1,icell-1] = 0.999

    ## Cell self propulsion velocity:

    vac = np.zeros(2)
    vac[0] = 0.5
    vac[1] = -0.5
    
    # end if ncell = 2

  return ncell,phis,nccel,ccell

if __name__ == '__main__':
    pass
    
    ###################### ending function ######################


In [ ]:
## laplacian:

# This function generates the Laplace operator by using five-point stencil with periodic boundaries.

import scipy as sp
from scipy import linalg
from scipy import sparse
import numpy as np
import math as m
    
def laplacian(Nx,Ny,dx,dy):

    format('long')
    nxny=Nx*Ny # Total number of grid points in the simulation cell

    ## Generate Laplace operator for five-point stencil:
    r=np.zeros(shape=(1,Nx))
    r[:,:2] = 2,-1
    T=sp.linalg.toeplitz(r) # symmetric Toeplitz matrix where: If r is a real vector, then r defines the first row of the matrix.
    E=sp.sparse.eye(Nx) # Sparse matrix (nx X nx) with ones on diagonal
    grad= -(sparse.kron(T,E) + sparse.kron(E,T)) # Computes the Kronecker product, a composite array made of blocks of the second array scaled by the first (POR AQUI ANDA EL FALLO)
    
    ## Modify the stencil for periodic boundaries:  
    for i in range(1,Nx+1):
        ii = (i-1)*Nx + 1
        jj = ii + Nx - 1
        grad[ii-1,jj-1]=1.0
        grad[jj-1,ii-1]=1.0
        
        kk = nxny - Nx + i
        grad[i-1,kk-1]=1.0
        grad[kk-1,i-1]=1.0
    
    ## Introduce dimensionality to the stencil:
    grad = grad/(dx*dy)
    
    return grad
    
if __name__ == '__main__':
    pass
    

In [ ]:
## vec2matx

# This function turns one-dimensional array to a square [N,N] matrix. 
# It is based on the built-in function vect2mat in Matlab/Octave and also utilizes two other built-in functions ceil and reshape.

import numpy as np
import math as m
    
def vec2matx(V=None,N=None,*args,**kwargs):

  format('long')
  V = np.transpose(V)
  V = np.ravel(V)
  R = m.ceil(len(V)/N)
  #M = np.reshape(np.transpose(V, (N,R), order='F'))
  M = np.reshape(V, (N,R), order='F').transpose() # FUNCIONA EN TEORIA!!??
  return M
    
if __name__ == '__main__':
    pass
    

In [ ]:
## free_energy_v2:

# This function evaluates the functional derivative of free energy function (Eq. 4.59)
# at all grid points for the current cell under consideration in the main program

import numpy as np

def free_energy_v2(Nx,Ny,icell,ncell,gamma,gamma_cell,kappa,kappa_cell,phi,phis):

  format('long')
  sum_phi = np.zeros(shape=(Nx,Ny)) # Initialize sum_phi for all grid points
  
  # Go over total number of cells in the simulation if jcell counter is not equal to icell add their value to sum_phi for all grid points
  for jcell in range(1,ncell+1):
    if (icell != jcell):
      sum_phi = sum_phi + kappa_cell*(phis[:,:,jcell-1])**2 

      #sum_phi = sum_phi + phis(np.arange(),np.arange(),jcell)**2 # RESULTADO DEL SMOP
      #sum_phi = sum_phi + phis(np.power( range(), range(),jcell,2)) # REVISAR ESTA LINEA
  
  # Calculate the functional derivative of the free energy for the current cell, the terms in the first bracket, Eq. 4.59, at all grid points
  dfdphi = gamma_cell*phi*(1.0-phi)*(1.0-2.0*phi) + 2.0*phi*sum_phi
  
  return dfdphi
    
if __name__ == '__main__':
    pass

In [ ]:
## gradient_mat:

## This function calculates the one-dimensional gradients for x and y directions of a given matrix.
## Function utilizes the built-in function gradient in Matlab/Octave, returns the values that are adjusted for grid spacing and the periodic boundaries
    
def gradient_mat(matx,Nx,Ny,dx,dy):

    format('long')
    
    matdx,matdy= np.gradient(matx)

    # for periodic boundaries:

    matdx[:Nx,0] = 0.5*(matx[:Nx,1] - matx[:Nx,Nx-1])
    matdx[:Nx,Nx-1] = 0.5*(matx[:Nx,0] - matx[:Nx,Nx-2])

    matdy[0,:Ny] = 0.5*(matx[1,:Ny] - matx[Ny-1,:Ny])
    matdy[Ny-1,:Ny] = 0.5*(matx[0,:Ny] - matx[Ny-2,:Ny])

    matdx = 2.0*matdx/dx
    matdy = 2.0*matdy/dy
    
    return matdx,matdy
    
if __name__ == '__main__':
    pass
    

In [ ]:
## write_vtk_grid_values

# This function writes the grid point values to a vtk file to generate the contour plots to be viewed by using Paraview.
# This function and its slightly modified versions are used throughout the book for visualization and animation of the results.
# Although, Matlab and Octave have impressive graphic routines; however, Paraview has more powerful features for visualization and spatial and temporal analysis of the data.

def write_vtk_grid_values(Nx=None,Ny=None,dx=None,dy=None,istep=None,data1=None,*args,**kwargs):

  format('long')
  #-- open output file
  
  fname = open("time_%d.vtk" % (istep), "w+") # Creating the file
  out = open("time_%d.vtk" % (istep), "a") # Appending text in the file

  nz = 1
  
  npoin = Nx*Ny*nz

  # start writing ASCII VTK file:
  
  # header of VTK file
  
  out.write("# vtk DataFile Version 2.0\n")
  out.write("time_10.vtk\n")
  out.write("ASCII\n")
  out.write("DATASET STRUCTURED_GRID\n")

  #--- coords of grid points:

  out.write("DIMENSIONS %5d %5d %5d\n" % (Nx,Ny,nz))

  out.write("POINTS %7d float\n" % (npoin))
  for i in range(1,Nx+1):
    for j in range(1,Ny+1):
      x =(i-1)*dx
      y =(j-1)*dy
      z = 0.0
      
      out.write("%14.6e %14.6e %14.6e\n" % (x,y,z))

  #--- write grid point values:

  out.write("POINT_DATA %5d\n" % (npoin))
  out.write("SCALARS CON float 1\n")
  out.write("LOOKUP_TABLE default\n")

  for i in range(1,Nx+1):
    for j in range(1,Ny+1):
      ii=(i-1)*Nx+j
      
      out.write("%14.6e\n" % (data1[i-1,j-1]))

  out.close()
  return
    
if __name__ == '__main__':
  pass
    

## Main Script: FINITE-DIFFERENCE PHASE-FIELD CODE FOR CELL-DYNAMICS

### Original code:

In [ ]:
############################################################
### FINITE-DIFFERENCE PHASE-FIELD CODE FOR CELL-DYNAMICS ###
############################################################

## This program solves the phase-field modeling of the cell dynamics with finite difference algorithm

# get initial wall time:
#time0 = datetime.now() # Current date and time
time0 = time.time() # ANOTHER WAY
format('long')

# Simulation cell parameters:

Nx = 200 # Number of grid points in the x-direction
Ny = 200 # Number of grid points in the y-direction
NxNy = Nx*Ny # Total number of grid points in the simulation cell

dx = 1.0 # Grid spacing between two grid points in the x-direction
dy = 1.0 # Grid spacing between two grid points in the y-direction

# Time integration parameters:
nstep = 3000 # Number of time integration steps
nprint = 50 # Output frequency to write the results to file
dtime = 5e-03 # Time increment for the numerical integration

# Material specific parameters:

### POLY-CELL SIMULATION ###
#ncell = 80 # number of cells
#R = 12.0  # radius of the cells
# For poly-cell simulations set the desired number of cells and radius of the cells (in grid points).


### 2-CELL SIMULATION ###
ncell = 2 # number of cells
R = 5  # radius of the cells
# For two-cell simulations set the ncell = 2 and radius of the cells (in grid points).


### Setting the values of the parameters ###
gamma = 5.0 # parameter that controls the elasticity of the cells
lamda = 7.0 # width of the boundary of any cell
kappa = 60.0 # free energy function (¿?¿¿?)
mu = 40.0 # energetic costs associated to change in cell area while keeping its volume approximately constant
kisa = 1.5e03 # friction between the cells and the surrounding liquid environment


### Pre-calculate some of the expressions as constants ###
pix = 4.0*atan(1.0) # 45º?? --> 3,1416 o pi, so 45º en radianes
const1 = 30.0/lamda**2
const2 = 2.0*mu/(pix*R**2)
const3 = 60.0*kappa/(lamda**2*kisa)

# Preparing initial microstructure:

ncell,phis,vac,nccel,ccell= micro_poly_cell(Nx,Ny,ncell,R)

# Getting Laplacian templet:

laplacian = laplacian(Nx,Ny,dx,dy)

# Time Evolution
for istep in range (1,nstep+1):
  # After 500 time steps, increase the initial time increment to 0.01
  if istep >= 500:
    dtime == 1.0e-2
  
  ### Looping over each cell in the simulation:
  for icell in range (1,ncell+1):
    # Cell elasticity parameter (Assign the cell elasticity parameter depending on
    # its character, determined by ccell): 
    gamma_cell = gamma
    for iccel in range (1,nccel+1):
      if icell == ccell[iccel-1]:
        gamma_cell = 0.5*gamma

    # Assign cells:
    phi = phis[ :, :,icell-1]
    
    # Calculate the laplacian and gradient terms:
    ## ' opertator (in Matlab) --> https://stackoverflow.com/questions/39885495/what-is-the-meaning-of-single-quote-in-matlab-and-how-to-change-it-to-python/39902500
    #phi2 = reshape(phi',NxNy,1);
    phi2 = np.reshape(phi.conj().transpose(), (NxNy,1), order="F") # REVISAR ESTOOOO: https://stackoverflow.com/questions/11892358/matlab-vs-python-reshape
    lap_phi2 = laplacian*phi2
    lap_phi = vec2matx(lap_phi2,Nx)

    # Gradients of phi:
    
    phidy,phidx = gradient_mat(phi,Nx,Ny,dx,dy)

    ## Second term:
    # derivative of free energy:
    
    dfdphi = free_energy_v2(Nx,Ny,icell,ncell,gamma_cell,kappa,phi,phis)
    
    term2 = -const1*dfdphi

    sum_phi = np.zeros(shape=(Nx,Ny))
    for jcell in range (1,ncell+1):
      if(icell != jcell):
        sum_phi = sum_phi + (phis[:,:,jcell-1])**2 # MI VERSION
        #sum_phi = sum_phi + phis(np.arange(),np.arange(),jcell)**2
        
    # volum integrations:
    
    dummy = np.zeros(shape=(Nx,Ny))
    dummy = phi ** 2
    vinteg = np.sum(np.sum(dummy))

    dummy = phi*phidx*sum_phi
    vintegx = np.sum(np.sum(dummy))

    dummy = phi*phidy*sum_phi
    vintegy = np.sum(np.sum(dummy))

    ## Third term:

    vinteg = vinteg - pix*(R**2)

    term3 = -const2*vinteg*phi

    # cell velocity:
    
    if (istep <= 200):
      vac_cell = 0.0
    else:
      vac_cell = vac[icell-1]
    
    # cell velocity vector:

    vnx = vac_cell + const3*vintegx

    vny = vac_cell + const3*vintegy

    if (ncell == 2): ##### ¡¡¡ POR ALGUN MOTIVO, LOS EJES ESTAN CAMBIADOS !!! (X se interpreta como Y, y viceversa) #####
      vnx = 0.0       # SHOULD BE "vac_cell"
      vny = vac_cell  # SHOULD BE "0.0"
    
    ## Fourth term:
    vnphi = vnx*phidx + vny*phidy

    # time integration:
    phi = phi + dtime*(gamma_cell*lap_phi + term2 + term3 -vnphi)

    # for small deviations:

    inrange = (phi >= 0.9999)
    phi[inrange] = 0.9999
    inrange = (phi < 1.0e-5)
    phi[inrange] = 0.0

    phis[:,:,icell-1] = phi

  # Print results:
  if ((np.mod(istep,nprint) == 0) or (istep == 1)):
    print("done step: %5d\n" % (istep))
    ## ESTA PARTE COMENTADA SE DEBERIA DE MODIFICAR
    #fname1 =sprintf('time_#d.out',istep)
    #out1 = fopen(fname1,'w')
    #for icell =1:ncell
    #fprintf(out1,'#cell No: #5d\n',icell)
    #for i=1:Nx
    #for j=1:Ny
    #fprintf(out1, '#5d #5d #14.6e\n' ,i,j,phis(i,j,icell))
    #end
    #end
    #end
    #fclose(out1)
    
    # write vtk file:

    phi1 = np.zeros(shape=(Nx,Ny))

    for jcell in range(1,ncell+1):
      add = 1.0

      for iccel in range(1,nccel+1):
        if (jcell == ccell[iccel-1]):
          add = 1.2
          #fprintf('jcell %5d %5d\n',jcell,ccell(iccel)); ## MATLAB LINE
      
      phi1 = phi1 + (phis[ :, :,jcell-1]**2 *add)


    write_vtk_grid_values(Nx,Ny,dx,dy,istep,phi1)

# end time:
end = time.time()

# Calculate compute time:
print(f"Runtime of the program is {end - time0} seconds")


/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:84: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


done step:     1

done step:    50

done step:   100

done step:   150

done step:   200

done step:   250

done step:   300

done step:   350

done step:   400

done step:   450

done step:   500

done step:   550

done step:   600

done step:   650

done step:   700

done step:   750

done step:   800

done step:   850

done step:   900

done step:   950

done step:  1000

done step:  1050

done step:  1100

done step:  1150

done step:  1200

done step:  1250

done step:  1300

done step:  1350

done step:  1400

done step:  1450

done step:  1500

done step:  1550

done step:  1600

done step:  1650

done step:  1700

done step:  1750

done step:  1800

done step:  1850

done step:  1900

done step:  1950

done step:  2000

done step:  2050

done step:  2100

done step:  2150

done step:  2200

done step:  2250

done step:  2300

done step:  2350

done step:  2400

done step:  2450

done step:  2500

done step:  2550

done step:  2600

done step:  2650

done step:  2700

done step:

### Code for cells constantly growing in Radius:

In [ ]:
######## Now, instead of moving, the cells will be GROWING IN RADIUS ########

############################################################
### FINITE-DIFFERENCE PHASE-FIELD CODE FOR CELL-DYNAMICS ###
############################################################

## This program solves the phase-field modeling of the cell dynamics with finite difference algorithm

# get initial wall time:
#time0 = datetime.now() # Current date and time
time0 = time.time() # ANOTHER WAY
format('long')

# Simulation cell parameters:

Nx = 200 # Number of grid points in the x-direction
Ny = 200 # Number of grid points in the y-direction
NxNy = Nx*Ny # Total number of grid points in the simulation cell

dx = 1.0 # Grid spacing between two grid points in the x-direction
dy = 1.0 # Grid spacing between two grid points in the y-direction

# Time integration parameters:
nstep = 2000 # Number of time integration steps
nprint = 50 # Output frequency to write the results to file
dtime = 5e-03 # Time increment for the numerical integration

# Material specific parameters:

### POLY-CELL SIMULATION ###
ncell = 20 # number of cells
R = 10.0  # radius of the cells
Rmax = 20.0 # Maximum radius of the cells
# For poly-cell simulations set the desired number of cells and radius of the cells (in grid points).


### 2-CELL SIMULATION ###
#ncell = 2 # number of cells
#R = 5  # radius of the cells
#Rmax = 30 # Maximum radius of the cells
# For two-cell simulations set the ncell = 2 and radius of the cells (in grid points).


### Setting the values of the parameters ###
gamma = 5.0   # parameter that controls the elasticity of the cells
lamda = 7.0   # width of the boundary of any cell
kappa = 60.0  # free energy function (¿?¿¿?)
mu = 40.0    # energetic costs associated to change in cell area while keeping its volume approximately constant
kisa = 1.5e03    # friction between the cells and the surrounding liquid environment


### Pre-calculate some of the expressions as constants ###
pix = 4.0*atan(1.0) # 45º?? --> 3,1416 o pi, so 45º en radianes
const1 = 30.0/lamda**2
const2 = 2.0*mu/(pix*R**2)
const3 = 60.0*kappa/(lamda**2*kisa)

# Preparing initial microstructure:

ncell,phis,nccel,ccell= micro_poly_cell(Nx,Ny,ncell,R)

# Getting Laplacian templet:

laplacian = laplacian(Nx,Ny,dx,dy)

# Time Evolution
for istep in range (1,nstep+1):
  # After 500 time steps, increase the initial time increment to 0.01
  if istep >= 500:
    dtime == 1.0e-2
  
  ### Looping over each cell in the simulation:
  for icell in range (1,ncell+1):
    # Cell elasticity parameter (Assign the cell elasticity parameter depending on its character, determined by ccell): 
    gamma_cell = gamma
    # The same for other parameters:
    kappa_cell = kappa
    #kisa_cell = kisa
    for iccel in range (1,nccel+1):
      if icell == ccell[iccel-1]:
        gamma_cell = 0.1*gamma # Less value, so SMOOTHER, if this value increases --> STIFFER
        kappa_cell = 2*kappa
        #kisa_cell = kisa*1e-2

    # Assign cells:
    phi = phis[ :, :,icell-1]
    
    # Calculate the laplacian and gradient terms:
    ## ' opertator (in Matlab) --> https://stackoverflow.com/questions/39885495/what-is-the-meaning-of-single-quote-in-matlab-and-how-to-change-it-to-python/39902500
    #phi2 = reshape(phi',NxNy,1);
    phi2 = np.reshape(phi.conj().transpose(), (NxNy,1), order="F") # REVISAR ESTOOOO: https://stackoverflow.com/questions/11892358/matlab-vs-python-reshape
    lap_phi2 = laplacian*phi2
    lap_phi = vec2matx(lap_phi2,Nx)

    # Gradients of phi:
    
    phidy,phidx = gradient_mat(phi,Nx,Ny,dx,dy)

    ## Second term:
    # derivative of free energy:
    
    dfdphi = free_energy_v2(Nx,Ny,icell,ncell,gamma,gamma_cell,kappa,kappa_cell,phi,phis)
    
    term2 = -const1*dfdphi

    sum_phi = np.zeros(shape=(Nx,Ny))

    for jcell in range (1,ncell+1):
      if (icell != jcell):
        # Checking whether each cell is smoother or stiffer
        if gamma_cell == gamma:
          # If they have the same stiffness, apply the same kappa value
          sum_phi = sum_phi + kappa*(phis[:,:,jcell-1])**2
        else:
          # If they have different stiffness, apply another kappa value
          sum_phi = sum_phi + kappa_cell*(phis[:,:,jcell-1])**2 
        
    # volum integrations:
    
    dummy = np.zeros(shape=(Nx,Ny))
    dummy = phi ** 2
    vinteg = np.sum(np.sum(dummy))

    dummy = phi*phidx*sum_phi
    vintegx = np.sum(np.sum(dummy))

    dummy = phi*phidy*sum_phi
    vintegy = np.sum(np.sum(dummy))

    ## Third term:

    vinteg = vinteg - pix*(R**2)

    term3 = -const2*vinteg*phi

    # CELL RADIUS INCREASE:  ##### RETOCAR ESTA PARTE #####

    ### MOVIDO ALGO MAS ABAJO ###

    # cell velocity vector: NOW IT SHOULD BE ZERO (vac_cell = 0 --> VnA, active component, or self-propulsion)

    vnx = const3*vintegx
    #vnx = 0

    vny = const3*vintegy
    #vny = 0

    #if (ncell == 2): ##### ¡¡¡ POR ALGUN MOTIVO, LOS EJES ESTAN CAMBIADOS !!! (X se interpreta como Y, y viceversa) #####
      #vnx = 0.0       # SHOULD BE "vac_cell"
      #vny = vac_cell  # SHOULD BE "0.0"
    
    ## Fourth term:
    vnphi = vnx*phidx + vny*phidy

    # time integration:
    phi = phi + dtime*(gamma_cell*lap_phi + term2 + term3 -vnphi)

    # for small deviations:

    inrange = (phi >= 0.9999)
    phi[inrange] = 0.9999
    inrange = (phi < 1.0e-5)
    phi[inrange] = 0.0

    phis[:,:,icell-1] = phi

    # CELL RADIUS INCREASE:  ##### RETOCAR ESTA PARTE #####
    
    #if (istep <= 200):
     # R = R
    #elif(istep > 200):
      #R += 0.001 ### THIS PART SHOULD BE CHANGED (Differential equation --> alpha - delta*R¿?)
      #if R >= Rmax:
        #break
        
    if(istep <= 200):
      R = R
    else:
      R += 5e-04
      if R >= Rmax:
        R = Rmax

  # Print results:
  if ((np.mod(istep,nprint) == 0) or (istep == 1)):
    print("done step: %5d\n" % (istep))
    print("R size: %5d\n" % (R))
    ## ESTA PARTE COMENTADA SE DEBERIA DE MODIFICAR
    #fname1 =sprintf('time_#d.out',istep)
    #out1 = fopen(fname1,'w')
    #for icell =1:ncell
    #fprintf(out1,'#cell No: #5d\n',icell)
    #for i=1:Nx
    #for j=1:Ny
    #fprintf(out1, '#5d #5d #14.6e\n' ,i,j,phis(i,j,icell))
    #end
    #end
    #end
    #fclose(out1)
    
    # write vtk file:

    phi1 = np.zeros(shape=(Nx,Ny))

    for jcell in range(1,ncell+1):
      add = 1.0

      for iccel in range(1,nccel+1):
        if (jcell == ccell[iccel-1]):
          add = 1.2
          #fprintf('jcell %5d %5d\n',jcell,ccell(iccel)); ## MATLAB LINE
      
      phi1 = phi1 + (phis[ :, :,jcell-1]**2 *add)


    write_vtk_grid_values(Nx,Ny,dx,dy,istep,phi1)

# end time:
end = time.time()

# Calculate compute time:
print(f"Runtime of the program is {end - time0} seconds")
print(f"Runtime of the program is {(end - time0)/60} minutes")


iteration done:   32

number of cell created:   20



/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:84: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


done step:     1

R size:    10

done step:    50

R size:    10

done step:   100

R size:    10

done step:   150

R size:    10

done step:   200

R size:    10

done step:   250

R size:    10

done step:   300

R size:    11

done step:   350

R size:    11

done step:   400

R size:    12

done step:   450

R size:    12

done step:   500

R size:    13

done step:   550

R size:    13

done step:   600

R size:    14

done step:   650

R size:    14

done step:   700

R size:    15

done step:   750

R size:    15

done step:   800

R size:    16

done step:   850

R size:    16

done step:   900

R size:    17

done step:   950

R size:    17

done step:  1000

R size:    18

done step:  1050

R size:    18

done step:  1100

R size:    19

done step:  1150

R size:    19

done step:  1200

R size:    19

done step:  1250

R size:    20

done step:  1300

R size:    20

done step:  1350

R size:    21

done step:  1400

R size:    21

done step:  1450

R size:    22

done step:

In [ ]:
## Check this: https://stackoverflow.com/questions/33091397/sparse-efficiency-warning-while-changing-the-column
## For the following warning: /usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:84: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient. self._set_intXint(row, col, x.flat[0])

### Code for cells growing in Radius (not constantly):

In [ ]:
######## Now, instead of moving, the cells will be GROWING IN RADIUS ########

############################################################
### FINITE-DIFFERENCE PHASE-FIELD CODE FOR CELL-DYNAMICS ###
############################################################

## This program solves the phase-field modeling of the cell dynamics with finite difference algorithm

# get initial wall time:
#time0 = datetime.now() # Current date and time
time0 = time.time() # ANOTHER WAY
format('long')

# Simulation cell parameters:

Nx = 200 # Number of grid points in the x-direction
Ny = 200 # Number of grid points in the y-direction
NxNy = Nx*Ny # Total number of grid points in the simulation cell

dx = 1.0 # Grid spacing between two grid points in the x-direction
dy = 1.0 # Grid spacing between two grid points in the y-direction

# Time integration parameters:
nstep = 2000 # Number of time integration steps
nprint = 50 # Output frequency to write the results to file
dtime = 5e-03 # Time increment for the numerical integration

# Material specific parameters:

### POLY-CELL SIMULATION ###
ncell = 30 # number of cells
R = 10.0  # radius of the cells
Rmax = 30.0 # Maximum radius of the cells
# For poly-cell simulations set the desired number of cells and radius of the cells (in grid points).


### 2-CELL SIMULATION ###
#ncell = 2 # number of cells
#R = 5  # radius of the cells
#Rmax = 30 # Maximum radius of the cells
# For two-cell simulations set the ncell = 2 and radius of the cells (in grid points).


### Setting the values of the parameters ###
gamma = 5.0 # parameter that controls the elasticity of the cells
lamda = 7.0 # width of the boundary of any cell
kappa = 60.0 # free energy function (¿?¿¿?)
mu = 40.0 # energetic costs associated to change in cell area while keeping its volume approximately constant
kisa = 1.5e03 # friction between the cells and the surrounding liquid environment

alpha = 100.0 # GROWTH RATE


### Pre-calculate some of the expressions as constants ###
pix = 4.0*atan(1.0) # 45º?? --> 3,1416 o pi, so 45º en radianes
const1 = 30.0/lamda**2
const2 = 2.0*mu/(pix*R**2)
const3 = 60.0*kappa/(lamda**2*kisa)

# Preparing initial microstructure:

ncell,phis,nccel,ccell= micro_poly_cell(Nx,Ny,ncell,R)

# Getting Laplacian templet:

laplacian = laplacian(Nx,Ny,dx,dy)

# Time Evolution
for istep in range (1,nstep+1):
  # After 500 time steps, increase the initial time increment to 0.01
  if istep >= 500:
    dtime == 1.0e-2
  
  ### Looping over each cell in the simulation:
  for icell in range (1,ncell+1):
    # Cell elasticity parameter (Assign the cell elasticity parameter depending on its character, determined by ccell): 
    gamma_cell = gamma
    # The same for other parameters:
    #kappa_cell = kappa
    #kisa_cell = kisa
    # Looping over all the soft cells generated:
    for iccel in range (1,nccel+1):
      # If the cell is soft (the cell id is the same as the soft cell id):
      if icell == ccell[iccel-1]:
        gamma_cell = 0.5*gamma # Less value, so SMOOTHER, if this value increases --> STIFFER
        #kappa_cell = 0.1*kappa # More value --> What should it mean? More interaction??
       # kisa_cell = 0.5*kisa # Less value --> LESS FRICTION WITH THE ENVIRONMENT

    # Assign cells:
    phi = phis[ :, :,icell-1]
    
    # Calculate the laplacian and gradient terms:
    ## ' opertator (in Matlab) --> https://stackoverflow.com/questions/39885495/what-is-the-meaning-of-single-quote-in-matlab-and-how-to-change-it-to-python/39902500
    #phi2 = reshape(phi',NxNy,1);
    phi2 = np.reshape(phi.conj().transpose(), (NxNy,1), order="F") # REVISAR ESTOOOO: https://stackoverflow.com/questions/11892358/matlab-vs-python-reshape
    lap_phi2 = laplacian*phi2
    lap_phi = vec2matx(lap_phi2,Nx)

    # Gradients of phi:
    
    phidy,phidx = gradient_mat(phi,Nx,Ny,dx,dy)

    ## Second term:
    # derivative of free energy:
    
    dfdphi = free_energy_v2(Nx,Ny,icell,ncell,gamma_cell,kappa,phi,phis)
    #dfdphi = free_energy_v2(Nx,Ny,icell,ncell,gamma,gamma_cell,kappa,kappa_cell,phi,phis)
    
    term2 = -const1*dfdphi

    sum_phi = np.zeros(shape=(Nx,Ny))

    for jcell in range (1,ncell+1):
      if(icell != jcell):
        sum_phi = sum_phi + (phis[:,:,jcell-1])**2 # MY VERSION
      #sum_phi = sum_phi + phis(np.arange(),np.arange(),jcell)**2

    ###If changing Kappa values, uncomment this:
   # for jcell in range (1,ncell+1):
     # if (icell != jcell):
        ## Checking whether each cell is smoother or stiffer
        ## gamma_cell is the stiffness value of the softest cells, and gamma is the value of the stiffest cells
       # if gamma_cell == gamma: 
          ## If the cell is stiff, apply the following (one kappa value):
          #sum_phi = sum_phi + kappa*(phis[:,:,jcell-1])**2
        #else:
          ## If not (== the cell is soft), apply another kappa value
          #sum_phi = sum_phi + kappa_cell*(phis[:,:,jcell-1])**2 
        
    # volum integrations:
    
    dummy = np.zeros(shape=(Nx,Ny))
    dummy = phi ** 2
    vinteg = np.sum(np.sum(dummy))

    dummy = phi*phidx*sum_phi
    vintegx = np.sum(np.sum(dummy))

    dummy = phi*phidy*sum_phi
    vintegy = np.sum(np.sum(dummy))

    ## Third term:

    vinteg = vinteg - pix*(R**2)

    term3 = -const2*vinteg*phi


    # cell velocity vector: NOW IT SHOULD BE ZERO (vac_cell = 0 --> VnA, active component, or self-propulsion)

    vnx = const3*vintegx
    #vnx = 0

    vny = const3*vintegy
    #vny = 0

    #if (ncell == 2): ##### ¡¡¡ POR ALGUN MOTIVO, LOS EJES ESTAN CAMBIADOS !!! (X se interpreta como Y, y viceversa) #####
      #vnx = 0.0       # SHOULD BE "vac_cell"
      #vny = vac_cell  # SHOULD BE "0.0"
    
    ## Fourth term:
    vnphi = vnx*phidx + vny*phidy

    # time integration:
    phi = phi + dtime*(gamma_cell*lap_phi + term2 + term3 -vnphi)


    # for small deviations:

    inrange = (phi >= 0.9999)
    phi[inrange] = 0.9999
    inrange = (phi < 1.0e-5)
    phi[inrange] = 0.0

    phis[:,:,icell-1] = phi


    # CELL RADIUS INCREASE:  ##### RETOCAR ESTA PARTE #####
    
    #if (istep <= 200):
      #R = R
    #elif (istep > 200):
      #R += 0.001 ### THIS PART SHOULD BE CHANGED (Differential equation --> alpha - gamma*R¿?)
      #if R >= Rmax:
        #break
      #R += (alpha/(1 + R)) - (gamma*R/1000) ### THIS PART SHOULD BE CHANGED (Differential equation --> alpha - gamma*R¿?)
      #R += alpha - gamma*R

    if(istep <= 200):
      R = R
    else:
      R += (alpha - gamma*R)*1e-5 # FIRST OPTION
      #R += ((alpha/(1 + R)) - (gamma*R/10))*1e-3 # SECOND OPTION
      if R >= Rmax:
        R = Rmax

  # Print results:
  if ((np.mod(istep,nprint) == 0) or (istep == 1)):
    print("done step: %5d\n" % (istep))
    print("R size: %5d\n" % (R))
    ## ESTA PARTE COMENTADA SE DEBERIA DE MODIFICAR
    #fname1 =sprintf('time_#d.out',istep)
    #out1 = fopen(fname1,'w')
    #for icell =1:ncell
    #fprintf(out1,'#cell No: #5d\n',icell)
    #for i=1:Nx
    #for j=1:Ny
    #fprintf(out1, '#5d #5d #14.6e\n' ,i,j,phis(i,j,icell))
    #end
    #end
    #end
    #fclose(out1)
    
    # write vtk file:

    phi1 = np.zeros(shape=(Nx,Ny))

    for jcell in range(1,ncell+1):
      add = 1.0

      for iccel in range(1,nccel+1):
        if (jcell == ccell[iccel-1]):
          add = 1.2
          #fprintf('jcell %5d %5d\n',jcell,ccell(iccel)); ## MATLAB LINE
      
      phi1 = phi1 + (phis[ :, :,jcell-1]**2 *add)


    write_vtk_grid_values(Nx,Ny,dx,dy,istep,phi1)

# end time:
end = time.time()

# Calculate compute time:
print(f"Runtime of the program is {end - time0} seconds")
print(f"Runtime of the program is {(end - time0)/60} minutes")


iteration done:   63

number of cell created:   30



/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:84: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


done step:     1

R size:    10

done step:    50

R size:    10

done step:   100

R size:    10

done step:   150

R size:    10

done step:   200

R size:    10

done step:   250

R size:    10

done step:   300

R size:    11

done step:   350

R size:    12

done step:   400

R size:    12

done step:   450

R size:    13

done step:   500

R size:    13

done step:   550

R size:    14

done step:   600

R size:    14

done step:   650

R size:    14

done step:   700

R size:    15

done step:   750

R size:    15

done step:   800

R size:    15

done step:   850

R size:    16

done step:   900

R size:    16

done step:   950

R size:    16

done step:  1000

R size:    16

done step:  1050

R size:    17

done step:  1100

R size:    17

done step:  1150

R size:    17

done step:  1200

R size:    17

done step:  1250

R size:    17

done step:  1300

R size:    18

done step:  1350

R size:    18

done step:  1400

R size:    18

done step:  1450

R size:    18

done step:

## Downloading vtk files generated:

In [ ]:
## Downloading all files generated: https://stackoverflow.com/questions/50453428/how-do-i-download-multiple-files-or-an-entire-folder-from-google-colab
!zip -r /content/vtk_files.zip /content/*.vtk

from google.colab import files
files.download("/content/vtk_files.zip")


  adding: content/time_1000.vtk (deflated 92%)
  adding: content/time_100.vtk (deflated 91%)
  adding: content/time_1050.vtk (deflated 92%)
  adding: content/time_1100.vtk (deflated 92%)
  adding: content/time_1150.vtk (deflated 92%)
  adding: content/time_1200.vtk (deflated 92%)
  adding: content/time_1250.vtk (deflated 92%)
  adding: content/time_1300.vtk (deflated 92%)
  adding: content/time_1350.vtk (deflated 93%)
  adding: content/time_1400.vtk (deflated 93%)
  adding: content/time_1450.vtk (deflated 93%)
  adding: content/time_1500.vtk (deflated 93%)
  adding: content/time_150.vtk (deflated 91%)
  adding: content/time_1550.vtk (deflated 93%)
  adding: content/time_1600.vtk (deflated 93%)
  adding: content/time_1650.vtk (deflated 93%)
  adding: content/time_1700.vtk (deflated 93%)
  adding: content/time_1750.vtk (deflated 93%)
  adding: content/time_1800.vtk (deflated 93%)
  adding: content/time_1850.vtk (deflated 93%)
  adding: content/time_1900.vtk (deflated 93%)
  adding: conte

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!ls

sample_data    time_1700.vtk  time_2450.vtk  time_3400.vtk  time_4800.vtk
time_1000.vtk  time_1750.vtk  time_2500.vtk  time_3500.vtk  time_4900.vtk
time_100.vtk   time_1800.vtk  time_250.vtk   time_350.vtk   time_5000.vtk
time_1050.vtk  time_1850.vtk  time_2550.vtk  time_3600.vtk  time_500.vtk
time_1100.vtk  time_1900.vtk  time_2600.vtk  time_3700.vtk  time_50.vtk
time_1150.vtk  time_1950.vtk  time_2650.vtk  time_3800.vtk  time_550.vtk
time_1200.vtk  time_1.vtk     time_2700.vtk  time_3900.vtk  time_600.vtk
time_1250.vtk  time_2000.vtk  time_2750.vtk  time_4000.vtk  time_650.vtk
time_1300.vtk  time_200.vtk   time_2800.vtk  time_400.vtk   time_700.vtk
time_1350.vtk  time_2050.vtk  time_2850.vtk  time_4100.vtk  time_750.vtk
time_1400.vtk  time_2100.vtk  time_2900.vtk  time_4200.vtk  time_800.vtk
time_1450.vtk  time_2150.vtk  time_2950.vtk  time_4300.vtk  time_850.vtk
time_1500.vtk  time_2200.vtk  time_3000.vtk  time_4400.vtk  time_900.vtk
time_150.vtk   time_2250.vtk  time_300.vtk   time

In [ ]:
!rm *.vtk
!rm vtk_files.zip

!ls

sample_data
